# TAQ midpoint price data

En la implementacion del midpoint price, divido la tarea en dos partes. Primero obtengo los resultados para todos los eventos en un dia y luego con otra funcion los calculo para todos los segundos en el dia.

### midpoint price data por evento por dia

Esta funcion no tiene problema en cuanto al tiempo de ejecucion. Las operaciones que realiza son muy basicas. Sin embargo, mi implementacion anterior usando solo `numpy` se demoraba 0,020s por dia y ahora se demora 0.1s por dia utilizando `pandas`.

La implementacion con `numpy` la voy a nombrar `taq_midpoint_event_data_numpy` y la implementacion con `pandas` la voy a nombrar `taq_midpoint_event_data_pandas`.

En este caso no entiendo por que con `pandas` es mas lento. La unica diferencia que hay es que la funcion de `numpy` lee un archivo `pickle` con una tupla con arrays mientras que la funcion `pandas` lee un archivo `hdf5` con un `DataFrame` de `pandas`.

In [1]:
# Modules

import numpy as np
import pandas as pd
import pickle
import time

In [2]:
def taq_midpoint_event_data_numpy(ticker, year, month, day):
    """Computes the midpoint price of every event.

    Using the dayly TAQ data computes the midpoint price of every event in a
    day.
    For further calculations, the function returns the values for the time
    range from 9h40 to 15h50.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param year: string of the year to be analized (i.e '2008').
    :param month: string of the month to be analized (i.e '07').
    :param day: string of the day to be analized (i.e '07').
    :return: tuple -- The function returns a tuple with numpy arrays.
    """

    # Load data
    # TAQ data gives directly the quotes data in every second that there is
    # a change in the quotes
    time_q_, bid_q_, ask_q_, _, _ = pickle.load(open(
        f'pickle_dayly_data_{year}/TAQ_{ticker}_quotes_{year}{month}{day}.pickle', 'rb'))

    # Some files are corrupted, so there are some zero values that
    # does not have sense
    condition_1 = ask_q_ != 0.
    time_q = time_q_[condition_1]
    bid_q = bid_q_[condition_1]
    ask_q = ask_q_[condition_1]

    assert len(bid_q) == len(ask_q)

    midpoint = (bid_q + ask_q) / 2
    spread = ask_q - bid_q

    return (time_q, bid_q, ask_q, midpoint, spread)

In [3]:
def taq_midpoint_event_data_pandas(ticker, date):
    """Computes the midpoint price of every event.

    Using the dayly TAQ data computes the midpoint price of every event in a
    day.
    For further calculations, the function returns the values for the time
    range from 9h40 to 15h50.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param year: string of the year to be analized (i.e '2008').
    :param month: string of the month to be analized (i.e '07').
    :param day: string of the day to be analized (i.e '07').
    :return: tuple -- The function returns a tuple with numpy arrays.
    """

    date_sep = date.split('-')
    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    # Load data
    # TAQ data gives directly the quotes data in every second that there is
    # a change in the quotes
    data_quotes_event = pd.read_hdf(
        f'hdf5_dayly_data_{year}/taq_{ticker}_quotes_{date}.h5')

    # Some files are corrupted, so there are some zero values that
    # does not have sense
    data_quotes_event = data_quotes_event[data_quotes_event['Ask'] != 0]

    data_quotes_event['Midpoint'] = (data_quotes_event['Bid']
                                     + data_quotes_event['Ask']) / 2
    data_quotes_event['Spread'] = data_quotes_event['Ask'] \
        - data_quotes_event['Bid']

    return data_quotes_event

In [4]:
ticker = 'AAPL'
year = '2008'
month = '01'
day = '02'
date = f'{year}-{month}-{day}'

In [5]:
%%timeit
taq_midpoint_event_data_numpy(ticker, year, month, day)

21.2 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
taq_midpoint_event_data_pandas(ticker, date)

1.88 s ± 85.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### midpoint price data por segundo por dia

Los datos obtenidos por la anterior funcion, dan los midpoint prices para cada evento. La idea con la siguiente funcion es obtener un punto valor de midpoint price por segundo en cada dia. Dado que no todos los segundos tienen necesariamente un valor de midpoint price, se da a los segundo sin precio el valor del segundo anterior.

La implementacion con `numpy` la voy a nombrar `taq_midpoint_time_data_numpy` y la implementacion con `pandas` la voy a nombrar `taq_midpoint_time_data_pandas`.

En este caso, la funcion se demora mucho en el paso en el que tiene que hacer la busqueda de los valores de cada segundo. En la version de `pandas` solo haciendo la busqueda se demora la mitad del tiempo.

In [7]:
def taq_midpoint_time_data_numpy(ticker, date):
    """Computes the midpoint price of every second.

    Using the taq_midpoint_event_data function computes the midpoint price of
    every second. To fill the time spaces when nothing happens I replicate the
    last value calculated until a change in the price happens.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param date: string with the date of the data to be extracted
     (i.e. '2008-01-02').
    :return: numpy array.
    """

    date_sep = date.split('-')

    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    function_name = taq_midpoint_time_data_numpy.__name__

    try:
        # Calculate the values of the midpoint price for all the events
        (time_q, bid_q, ask_q,
         midpoint, spread) = taq_midpoint_event_data_numpy(ticker, year, month, day)

        # 34800 s = 9h40 - 57000 s = 15h50
        # Reproducing S. Wang values. In her results the time interval for the
        # midpoint is [34800, 56999]
        full_time = np.array(range(34800, 57000))

        # As there can be several values for the same second, we use the
        # last value of each second in the full time array as it behaves
        # quiet equal as the original input

        midpoint_last_val = 0. * full_time
        midpoint_last_val[-1] = midpoint[0]

        ask_last_val = 0. * full_time
        ask_last_val[-1] = ask_q[0]

        bid_last_val = 0. * full_time
        bid_last_val[-1] = bid_q[0]

        spread_last_val = 0. * full_time
        spread_last_val[-1] = spread[0]

        for t_idx, t_val in enumerate(full_time):

            condition = time_q == t_val

            if (np.sum(condition)):

                midpoint_last_val[t_idx] = midpoint[condition][-1]
                ask_last_val[t_idx] = ask_q[condition][-1]
                bid_last_val[t_idx] = bid_q[condition][-1]
                spread_last_val[t_idx] = spread[condition][-1]

            else:

                midpoint_last_val[t_idx] = midpoint_last_val[t_idx - 1]
                ask_last_val[t_idx] = ask_last_val[t_idx - 1]
                bid_last_val[t_idx] = bid_last_val[t_idx - 1]
                spread_last_val[t_idx] = spread_last_val[t_idx - 1]

        # There should not be 0 values in the midpoint array
        assert not np.sum(midpoint_last_val == 0)

        return midpoint_last_val

    except FileNotFoundError as e:
            print('No data')
            print(e)
            print()
            return None

In [8]:
def taq_midpoint_time_data_pandas_ori(ticker, date):
    """Computes the midpoint price of every second.

    Using the taq_midpoint_event_data function computes the midpoint price of
    every second. To fill the time spaces when nothing happens I replicate the
    last value calculated until a change in the price happens.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param date: string with the date of the data to be extracted
     (i.e. '2008-01-02').
    :return: numpy array.
    """

    date_sep = date.split('-')

    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    function_name = taq_midpoint_time_data_pandas_ori.__name__

    try:
        # Calculate the values of the midpoint price for all the events
        data_quotes_event = taq_midpoint_event_data_pandas(ticker, date)

        # 34800 s = 9h40 - 57000 s = 15h50
        # Reproducing S. Wang values. In her results the time interval for the
        # midpoint is [34800, 56999]
        full_time = np.array(range(34800, 57000))

        # As there can be several values for the same second, we use the
        # last value of each second in the full time array as it behaves
        # quiet equal as the original input
        set_data_time = np.array(list(set(data_quotes_event['Time'])))
        list_data_time = [0] * len(full_time)

        for t_idx, t_val in enumerate(full_time):
            condition = data_quotes_event['Time'] == t_val
            if (np.sum(condition)):
                
                data_dict = {'Time': data_quotes_event[condition].iloc[-1]['Time'],
                             'Midpoint': data_quotes_event[condition].iloc[-1]['Midpoint']}

                list_data_time[t_idx] = data_dict

            else:

                data_dict = {'Time': list_data_time[t_idx - 1]['Time'],
                             'Midpoint': list_data_time[t_idx - 1]['Midpoint']}

                list_data_time[t_idx] = data_dict

        data_quotes_time = pd.DataFrame(list_data_time, columns=['Time', 'Midpoint'])

        # The lengths of the time and the dataframe have to be the same
        assert len(full_time) == len(data_quotes_time['Time'])

        data_quotes_time['Time'] = full_time

        return data_quotes_time

    except FileNotFoundError as e:
        print('No data')
        print(e)
        print()
        return None

In [9]:
def taq_midpoint_time_data_pandas_for(ticker, date):
    """Computes the midpoint price of every second.

    Using the taq_midpoint_event_data function computes the midpoint price of
    every second. To fill the time spaces when nothing happens I replicate the
    last value calculated until a change in the price happens.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param date: string with the date of the data to be extracted
     (i.e. '2008-01-02').
    :return: numpy array.
    """

    date_sep = date.split('-')

    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    function_name = taq_midpoint_time_data_pandas_for.__name__
    
    try:
        # Calculate the values of the midpoint price for all the events
        data_quotes_event = taq_midpoint_event_data_pandas(ticker, date)

        # 34800 s = 9h40 - 57000 s = 15h50
        # Reproducing S. Wang values. In her results the time interval for the
        # midpoint is [34800, 56999]
        full_time = np.array(range(34800, 57000))
        midpoint = np.array(range(34800, 57000))

        t0 = time.time()
        for t_idx, t_val in enumerate(full_time):

            condition = data_quotes_event['Time'] == t_val
            if (np.sum(condition)):
                midpoint[t_idx] = data_quotes_event[condition].iloc[-1]['Midpoint']

        t1 = time.time()
        print(f'First loop {t1-t0}')

        t0 = time.time()
        for t_idx, t_val in enumerate(full_time):

            if (midpoint[t_idx] == 0):
                midpoint[t_idx] = midpoint[t_idx - 1]

        t1 = time.time()
        print(f'Second loop {t1-t0}')

        data_quotes_time = pd.DataFrame(columns=['Time', 'Midpoint'])
        data_quotes_time['Time'] = full_time
        data_quotes_time['Midpoint'] = midpoint

        return data_quotes_time

    except FileNotFoundError as e:
        print('No data')
        print(e)
        print()
        return None

In [10]:
%%timeit
taq_midpoint_time_data_numpy(ticker, date)

19.5 s ± 192 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
# En esta version, hay un solo for y un if dentro del for
taq_midpoint_time_data_pandas_ori(ticker, date)

2min 39s ± 2.38 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
# En esta version hay dos for
taq_midpoint_time_data_pandas_for(ticker, date)

First loop 141.59047603607178
Second loop 0.009029626846313477
First loop 142.71122193336487
Second loop 0.008977174758911133
First loop 146.88000965118408
Second loop 0.009099721908569336
First loop 143.41236209869385
Second loop 0.009290218353271484
First loop 141.14672875404358
Second loop 0.009501218795776367
First loop 138.35231924057007
Second loop 0.008942365646362305
First loop 135.26141262054443
Second loop 0.008853912353515625
First loop 138.24357104301453
Second loop 0.00905299186706543
2min 22s ± 3.59 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
